In [6]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the Black-Scholes price for a European option.
    
    Parameters:
    S : Current stock price
    K : Strike price
    T : Time to maturity (in years)
    r : Risk-free interest rate (annual)
    sigma : Volatility of the underlying asset (annual)
    option_type : 'call' for call option, 'put' for put option
    
    Returns:
    Price of the option
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

def option_greeks(S, K, T, r, sigma, option_type='call'):
   
    ### Derive and calculate Greeks using BSM model

    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    delta = norm.cdf(d1) if option_type == 'call' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    theta = - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2) if option_type == 'call' else - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)
    rho = K * T * np.exp(-r * T) * norm.cdf(d2) if option_type == 'call' else -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta, 'rho': rho}

def delta_hedge(S, K, T, r, sigma, option_type= 'call'):
   
    ### Calculate the number of stocks needed to hedge based on option delta

    greeks = option_greeks(S, K, T, r, sigma, option_type)
    delta = greeks['delta']
    if option_type == 'call':
        return -delta #to hedge the option, take the opposite delta position in stock
    else: return delta

def inflection_point(S, K, T, r):
    
    ### Calculate the inflection point of volatility with respect to option price
    
    m = S / (K * np.exp(-r * T))
    return np.sqrt(2 * abs(np.log(m)) / T)             

def option_vega(S, K, T, r, sigma):

    ### calculating d1 from black scholes
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    ### Calculate Vega
    vega = S * norm.pdf(d1) * np.sqrt(T)
    return vega

def implied_volatility(option_type, market_price, S, K, T, r):
    
    x0 = inflection_point(S, K, T, r)
    tol=10e-8
    
    def newton_raphson(S, K, T, r, market_price, option_type, x0, tol):
        delta = black_scholes_price(S, K, T, r, x0, option_type) - market_price
        
        while delta > tol:
            x0 = x0 - ((black_scholes_price(S, K, T, r, x0, option_type) - market_price) / option_vega(S, K, T, r, x0))
            delta = abs(black_scholes_price(S, K, T, r, x0, option_type) - market_price)
        return x0
    
    sig = newton_raphson(S, K, T, r, market_price, option_type, x0, tol)
    return sig*100   
    

In [25]:
S = 165 # Current spot price
K = 170  # Strike price
T = 0.5    # Time to maturity in years
r = 0.03 # Risk-free interest rate
sigma = 0.1  # Volatility estimate
market_price = 11 # Market price of option
option_type = 'put' # Either 'call' or 'put'

option_price = black_scholes_price(S, K, T, r, sigma, option_type)
print("Option Price:", option_price)

greeks = option_greeks(S, K, T, r, sigma, option_type)
print("Option Greeks:", greeks)

hedge_position = delta_hedge(S, K, T, r, sigma, option_type)
print("Delta Hedge Position (shares to hold):", hedge_position)

implied_vol = implied_volatility(option_type, market_price, S, K, T, r)
print("Implied Volatility:", implied_vol, "%")

Option Price: 6.025934557471615
Option Greeks: {'delta': -0.5693412475082378, 'gamma': 0.03367549378003524, 'vega': 45.84076590807296, 'theta': -1.5850593789173706, 'rho': -49.98362019816543}
Delta Hedge Position (shares to hold): -0.5693412475082378
Implied Volatility: 20.73720064469495 %
